# Movie Recommender Systems

This notebook implements various movie recommendation approaches:
1. Popularity-based
2. Content-based Filtering
3. Collaborative Filtering
4. Matrix Factorization
5. Hybrid Approach

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
import torch
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading and Preprocessing

In [2]:
# Read the datasets
movies_df = pd.read_csv('ml-latest-small/movies.csv')
ratings_df = pd.read_csv('ml-latest-small/ratings.csv')
tags_df = pd.read_csv('ml-latest-small/tags.csv')

# Merge tags for each movie
tags_grouped = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
movies_with_tags = pd.merge(movies_df, tags_grouped, on='movieId', how='left')
movies_with_tags['tag'] = movies_with_tags['tag'].fillna('')

## 2. Popularity-based Recommender

In [3]:
def popularity_recommender(n_recommendations=10):
    # Calculate mean rating and number of ratings for each movie
    movie_stats = ratings_df.groupby('movieId').agg({
        'rating': ['count', 'mean']
    }).reset_index()
    
    # Rename columns
    movie_stats.columns = ['movieId', 'rating_count', 'rating_mean']
    
    # Filter movies with minimum number of ratings (e.g., 100)
    popular_movies = movie_stats[movie_stats['rating_count'] >= 100]
    
    # Sort by rating mean and count
    popular_movies = popular_movies.sort_values(['rating_mean', 'rating_count'], ascending=[False, False])
    
    # Get movie titles
    recommendations = pd.merge(popular_movies, movies_df, on='movieId')
    
    return recommendations[['title', 'rating_mean', 'rating_count']].head(n_recommendations)

## 3. Content-based Filtering

In [4]:
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Create document-term matrix
tfidf_matrix = tfidf.fit_transform(movies_with_tags['tag'])

# Perform dimensionality reduction using TruncatedSVD
svd = TruncatedSVD(n_components=100)
latent_matrix_1 = svd.fit_transform(tfidf_matrix)

def content_based_recommender(movie_title, n_recommendations=10):
    # Get movie index
    movie_idx = movies_with_tags[movies_with_tags['title'] == movie_title].index[0]
    
    # Calculate similarity scores
    movie_similarities = cosine_similarity([latent_matrix_1[movie_idx]], latent_matrix_1)[0]
    
    # Get top similar movies
    similar_indices = movie_similarities.argsort()[::-1][1:n_recommendations+1]
    
    return movies_with_tags.iloc[similar_indices][['title', 'genres']]

## 4. Collaborative Filtering

In [5]:
# Create user-movie matrix
user_movie_matrix = ratings_df.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Perform SVD for collaborative filtering
svd_collab = TruncatedSVD(n_components=100)
latent_matrix_2 = svd_collab.fit_transform(user_movie_matrix)

def collaborative_recommender(user_id, n_recommendations=10):
    # Get user's latent features
    user_idx = user_movie_matrix.index.get_loc(user_id)
    user_features = latent_matrix_2[user_idx]
    
    # Calculate predicted ratings
    predicted_ratings = np.dot(user_features, svd_collab.components_)
    
    # Get movies user hasn't rated
    user_ratings = user_movie_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index
    
    # Get top recommendations
    movie_indices = (-predicted_ratings[unrated_movies]).argsort()[:n_recommendations]
    recommended_movies = movies_df[movies_df['movieId'].isin(unrated_movies[movie_indices])]
    
    return recommended_movies[['title', 'genres']]

## 5. Matrix Factorization using Surprise

In [6]:
# Prepare data for Surprise
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

# Split data
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Train SVD model
svd_model = SVD(n_factors=100, random_state=42)
svd_model.fit(trainset)

def matrix_factorization_recommender(user_id, n_recommendations=10):
    # Get all movies
    all_movies = movies_df['movieId'].unique()
    
    # Get user's rated movies
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].unique()
    
    # Get unrated movies
    unrated_movies = np.setdiff1d(all_movies, rated_movies)
    
    # Predict ratings for unrated movies
    predictions = [svd_model.predict(user_id, movie_id) for movie_id in unrated_movies]
    
    # Sort predictions
    sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:n_recommendations]
    
    # Get recommended movies
    recommended_movies = movies_df[movies_df['movieId'].isin([pred.iid for pred in sorted_predictions])]
    
    return recommended_movies[['title', 'genres']]

## 6. Hybrid Recommender

In [7]:
def hybrid_recommender(user_id, movie_title, n_recommendations=10):
    # Get recommendations from each method
    content_recs = content_based_recommender(movie_title, n_recommendations)
    collab_recs = collaborative_recommender(user_id, n_recommendations)
    mf_recs = matrix_factorization_recommender(user_id, n_recommendations)
    
    # Combine recommendations
    all_recs = pd.concat([
        content_recs.assign(method='content'),
        collab_recs.assign(method='collaborative'),
        mf_recs.assign(method='matrix_factorization')
    ])
    
    # Remove duplicates and sort by frequency of appearance
    final_recs = all_recs.groupby('title').agg({
        'genres': 'first',
        'method': lambda x: ', '.join(x)
    }).reset_index()
    
    final_recs['recommendation_strength'] = final_recs['method'].str.count(',')
    
    return final_recs.sort_values('recommendation_strength', ascending=False).head(n_recommendations)

## 7. Example Usage

In [8]:
# Example usage
print("Popular Movies:")
print(popularity_recommender())

print("\nContent-based Recommendations for 'Toy Story (1995)':")
print(content_based_recommender('Toy Story (1995)'))

print("\nCollaborative Filtering Recommendations for user 1:")
print(collaborative_recommender(1))

print("\nMatrix Factorization Recommendations for user 1:")
print(matrix_factorization_recommender(1))

print("\nHybrid Recommendations for user 1 and 'Toy Story (1995)':")
print(hybrid_recommender(1, 'Toy Story (1995)'))

Popular Movies:
                              title  rating_mean  rating_count
0  Shawshank Redemption, The (1994)     4.429022           317
1             Godfather, The (1972)     4.289062           192
2                 Fight Club (1999)     4.272936           218
3    Godfather: Part II, The (1974)     4.259690           129
4              Departed, The (2006)     4.252336           107
5                 Goodfellas (1990)     4.250000           126
6                 Casablanca (1942)     4.240000           100
7           Dark Knight, The (2008)     4.238255           149
8        Usual Suspects, The (1995)     4.237745           204
9        Princess Bride, The (1987)     4.232394           142

Content-based Recommendations for 'Toy Story (1995)':
                                 title  \
1757              Bug's Life, A (1998)   
8695  Guardians of the Galaxy 2 (2017)   
2355                Toy Story 2 (1999)   
8123                Croods, The (2013)   
2077            Iron Giant

IndexError: index 25746 is out of bounds for axis 0 with size 9724